_Task1: Run several analogy solving models with several different representations on the benchmarking analogy dataset and report your findings._ 

Focus on the following questions:

1. Is the choice of the analogy model important? Which representations work better with which analogy models?
    
    Yes. The choice of the analogy model will affect not only computation complexity but also recall. 
    
    For word2vec model, multiplication model works better than additional model.
    
    For GloVe model, when dimensions are lower than 300, additional model is better than multiplication model. When dimension equals to 300,
    multiplication model is better.
    
    I didn't implement the direction model.
    
2. Is dimensionality of the representation important when using GloVe vectors?
    
    Yes. When dimensions are relatively high, the performance (recalll) of the same analogy model increases because it can keep more features
    or hidden information of words.
    
3. What is the computational complexity of the analogy models given the pre-trained vectors?

    I introduced restrict_vocab to reduce the computation complexity. We only need to search analogy word from restric_vocab words 
    instead of all words in the dictionary. 
    
    Set the number of questions is N, the number of restricted vocabulary is n (equals to restrict_vocab), the dimension of each word vector 
    is (1, d), the number of all words in dictionary is n_all.
    Assume the sorting function doesn't need any time and space.
    
    For addition model, the time complexity is O(N\*d\*n), the space complexity is O(n_all\*d)
    
    For multiplication model, the time complexity is O(N\*d\*n\*3), the space complexity is O(n_all\*d) . The computation of this model is 
    longer than addition model because it needs calculate inner product for 3 times. For each question, after doing the inner product, 
    there will be 3 word vectors with dimension (1, n). The numerical mutiplication of these 3 vectors need extra time O(N\*n) which is 
    much shorter than O(N\*d\*n\*3) so we can ignore it.
    
    The extra space for computation is about O(1, n) which is much smaller than O(n_all\*d) so we can also ignore it. 
    O(n_all\*d) is the space for storing the pre-trained model. 

4. What are the typical errors?
    
    One error is from the testing set. It says London is capital of England but actually it is UK or Britain. So there are about 200 false negative is about that. The country name has many representations. They have the same meaning but they don't have exactly the same string. This is the reason that cause false negative. 
    
    Another error is the given word is not in the vocabulary (capital letter matters). So it doesn't have a vector to present itself.

In [1]:
from gensim import models, matutils, utils
import numpy as np
import smart_open
import os
import logging
import sys
import random
from six import iteritems
logger = logging.getLogger()
logger.setLevel(logging.INFO)
# Get logging information

In [2]:
def glove2word2vec(glove_filename):
    """
    Convert glove file to word2vec format
    The only difference between word2vec format and glove is
    word2vec provides number of lines and dimension of word vectors
    in first line
    This is a short cut for given 4 glove file:
        "glove.6B.50d.txt", "glove.6B.100d.txt", "glove.6B.200d.txt", "glove.6B.300d.txt"
    Other format of glove filename needs other modification of getFirstLineInfo    
    """
    def getFirstLineInfo(glove_filename):
        """
        Calculate the number of lines and dimensions of word vector
        """
        num_lines = sum(1 for line in smart_open.smart_open(glove_filename))
        # the file name of glove file contains number of dimensions so we can extract that from file name
        dims = glove_filename.split('.')[2].split('d')[0]
        return num_lines, dims
    
    def addFirstLine(glove_filename, word2vec_filename, first_line_info):
        """
        Add information of number of lines and dimensions into the first line
        """
        with smart_open.smart_open(glove_filename, 'rb') as infile:
            with smart_open.smart_open(word2vec_filename, 'wb') as outfile:
                outfile.write(str(first_line_info) + '\n')
                for line in infile:
                    outfile.write(line)
        return word2vec_filename
    
    word2vec_filename = glove_filename[:-3] + "word2vec.txt"
    if os.path.isfile(word2vec_filename):
        model = models.word2vec.Word2Vec.load_word2vec_format(word2vec_filename)
    else:
        num_lines, dims = getFirstLineInfo(glove_filename)
        first_line = "{} {}".format(num_lines, dims)
        model_file = addFirstLine(glove_filename, word2vec_filename, first_line)
        model = models.word2vec.Word2Vec.load_word2vec_format(model_file)
    
    # normalize all word vectors
    model.init_sims(replace = True)
    return model

In [3]:
def findAnalogy_model1(a, b, c, model, restrict_vocab = None): 
    """
    addition model
    a to b is c to d. a, b, c, d are all words. 
    d = argmax(cos(d', c-a+b)). 
    
    bonus:
        If you want to find the most corrected word of a given word, you can use
        findAnalogy_model1("empty", "empty", "given_word", model)
        yes. I know you find out the first two "empty" can be replaced by any word but these two must be exactly the same word.
    """
    mixedNormVec = None
    all_words = set()
    for word in [a, b, c]:
        if not word in model.vocab:
            raise KeyError("word '%s' not in vocabulary" % word)
        else:
            all_words.add(model.vocab[word].index)
    # normalize the result of b + c - a. prepare for computing cosine similarity
    mixedNormVec = matutils.unitvec(model[b] + model[c] - model[a]).astype(np.float32)
    
    limited = model.syn0norm if restrict_vocab is None else model.syn0norm[:restrict_vocab]
    # calculate the cosine similarity between all words (d') and c-a+b
    sims = np.dot(limited, mixedNormVec)
    # find 15 best result which is the highest similarity score
    # it is possible that finding the same word as a or b or c 
    # so we need to give some space for other possible words
    best = matutils.argsort(sims, topn = 15, reverse=True)
    # ignore words from the input and words with special symbols
    result = [(model.index2word[sim], float(sims[sim])) for sim in best if (sim not in all_words) and (("_" not in model.index2word[sim]) or \
                                                                                                       (("\\") not in model.index2word))]
    return result[0]


In [4]:
def cr(arr):
    """
    cr for Convert Range
    arr is a numpy array
    """
    return ((arr + 1.0) / 2.0)

In [5]:
def findAnalogy_model2(a, b, c, model, restrict_vocab = None):
    """
    multiplication model
    a to b is c to d. d = argmax(cos(d',c)*cos(d',b)/(cos(d',a)+e))
    e = 0.001 to avoid division by zero
    """
    all_words = set()
    for word in [a, b, c]:
        if not word in model.vocab:
            raise KeyError("word '%s' not in vocabulary" % word)
        else:
            all_words.add(model.vocab[word].index)

    limited = model.syn0norm if restrict_vocab is None else model.syn0norm[:restrict_vocab]
    sims = (cr(np.dot(limited, model.syn0norm[model.vocab[c].index])) * cr(np.dot(limited, model.syn0norm[model.vocab[b].index]))) / \
        (cr(np.dot(limited, model.syn0norm[model.vocab[a].index])) + 0.001)
    best = matutils.argsort(sims, topn = 15, reverse=True)
    # ignore words from the input
    result = [(model.index2word[sim], float(sims[sim])) for sim in best if (sim not in all_words) and (("_" not in model.index2word[sim]) or \
                                                                                                       (("\\") not in model.index2word))]
    return result[0]

In [6]:
def findAnalogyPerLine(line, r_model, a_model, currentCount, restrict_vocab = None, caps = False):
    """
    For each question, I have a, b, c, expected. I need to guess d from a, b, c and compare that with expected
    currentCount is a list contains two elements, first one is number of current correct answer
    the second one is the number of all questions we tested so far
    """
    if line[0] == ':':
        return currentCount
    else:
        if caps:
            a, b, c, expected = line.split()
        else:
            a, b, c, expected = [word.lower() for word in line.split()]
        currentCount[1] += 1
        if a_model == 1:
            d = findAnalogy_model1(a, b, c, r_model, restrict_vocab = restrict_vocab)
        elif a_model == 2:
            d = findAnalogy_model2(a, b, c, r_model, restrict_vocab = restrict_vocab)
        else:
            raise ValueError("invalid analogy model")
        if d[0] == expected:
            currentCount[0] += 1
    return currentCount        

In [7]:
def recallOfAnalogyModel(questions, r_model, a_model, restrict_vocab = 30000, caps = False):
    """
    questions are file path for the test file. In this case, 'questions-words.txt'
    r_model is representation model of word vector (word2vec, GloVe)
    a_model is analogy model (1, 2). 1 stands for findAnalogy_model1, 2 stands for findAnalogy_model2
    restrict the number of words to be compared to increase the speed. 
    caps determines if we need to keep all capital letters. True for keeping, False for transforming into lower case
    """
    # currentCount is a list contains two elements, first one is number of current correct answer
    currentCount = [0, 0]
    random.seed(0) # set a random seed to reproduce the random numbers
    # randomly select about 1/20 of questions to limit the computation time
    rand_lines =[random.randint(i*20, (i + 1) * 20) for i in range(19558 // 20)]
    # the words with lower count will not be considered as an option for answer of analogy question. They will be kicked out of ok_vocab
    ok_vocab = dict(sorted(iteritems(r_model.vocab), key=lambda item: -item[1].count)[:restrict_vocab])
    with open(questions, 'r') as ifile:
        for line_no, line in enumerate(utils.smart_open(questions)):
            if line_no in rand_lines:
                if line[0] != ":":
                    if caps:
                        a, b, c, expected = line.split()
                    else:
                        # transform all letters into lower case letters
                        a, b, c, expected = [word.lower() for word in line.split()]
                    if a not in ok_vocab or b not in ok_vocab or c not in ok_vocab or expected not in ok_vocab:
                        logger.info("skipping line #%i with OOV words: %s" % (line_no, line.strip()))
                        continue
                    else:
                        findAnalogyPerLine(line, r_model, a_model, currentCount, restrict_vocab = restrict_vocab, caps = caps)
                        
                if currentCount[1]  % 200 == 0:
                    loginfo = "correct pair:" + str(currentCount[0]) + ", total pair:" + str(currentCount[1])
                    logger.info(loginfo)
    
    recall = float(currentCount[0]) / float(currentCount[1])
    return float('%.4f'% recall)


In [8]:
# Load pre-trained representation model. Whether load those model at the same time depends on the space of your RAM

# Load pre-trained word2vec model from disk
word2vec_model = models.word2vec.Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)

# word2vec_model.vocab: 3000000  words for word2vec_model.
# each word are represented as a vector with 300 terms
# word2vec_model.syn0: matrix for the model
# word2vec_model.syn0.shape: check the shape of this matrix
# Normalize all vectors in this model. 
# So we can use dot product to calculate cosine similarity which is more efficient
word2vec_model.init_sims(replace = True)
# The normalized vectors are stored in model.syn0 and model.syn0norm. These are the same.

INFO:gensim.models.word2vec:loading projection weights from GoogleNews-vectors-negative300.bin
INFO:gensim.models.word2vec:loaded (3000000L, 300L) matrix from GoogleNews-vectors-negative300.bin
INFO:gensim.models.word2vec:precomputing L2-norms of word weight vectors


In [11]:
print "recall of word2vec & addition model:", \
recallOfAnalogyModel('questions-words.txt', word2vec_model, 1, restrict_vocab = 200000, caps = True)

INFO:root:skipping line #965 with OOV words: Baghdad Iraq Funafuti Tuvalu
INFO:root:skipping line #1041 with OOV words: Bamako Mali Funafuti Tuvalu
INFO:root:skipping line #1231 with OOV words: Belgrade Serbia Funafuti Tuvalu
INFO:root:skipping line #1535 with OOV words: Budapest Hungary Funafuti Tuvalu
INFO:root:skipping line #2075 with OOV words: Funafuti Tuvalu Jakarta Indonesia
INFO:root:skipping line #2082 with OOV words: Funafuti Tuvalu Kingston Jamaica
INFO:root:skipping line #2104 with OOV words: Funafuti Tuvalu Niamey Niger
INFO:root:skipping line #2373 with OOV words: Islamabad Pakistan Nuuk Greenland
INFO:root:skipping line #2639 with OOV words: Kigali Rwanda Nuuk Greenland
INFO:root:skipping line #2753 with OOV words: Lilongwe Malawi Nuuk Greenland
INFO:root:skipping line #3209 with OOV words: Minsk Belarus Nuuk Greenland
INFO:root:skipping line #3440 with OOV words: Nairobi Kenya Paramaribo Suriname
INFO:root:skipping line #3633 with OOV words: Nuuk Greenland Quito Ecuador

 recall of word2vec & addition model: 0.7441


In [12]:
print "recall of word2vec & mutiplication model:", \
recallOfAnalogyModel('questions-words.txt', word2vec_model, 2, restrict_vocab = 200000, caps = True)

INFO:root:skipping line #965 with OOV words: Baghdad Iraq Funafuti Tuvalu
INFO:root:skipping line #1041 with OOV words: Bamako Mali Funafuti Tuvalu
INFO:root:skipping line #1231 with OOV words: Belgrade Serbia Funafuti Tuvalu
INFO:root:skipping line #1535 with OOV words: Budapest Hungary Funafuti Tuvalu
INFO:root:skipping line #2075 with OOV words: Funafuti Tuvalu Jakarta Indonesia
INFO:root:skipping line #2082 with OOV words: Funafuti Tuvalu Kingston Jamaica
INFO:root:skipping line #2104 with OOV words: Funafuti Tuvalu Niamey Niger
INFO:root:skipping line #2373 with OOV words: Islamabad Pakistan Nuuk Greenland
INFO:root:skipping line #2639 with OOV words: Kigali Rwanda Nuuk Greenland
INFO:root:skipping line #2753 with OOV words: Lilongwe Malawi Nuuk Greenland
INFO:root:skipping line #3209 with OOV words: Minsk Belarus Nuuk Greenland
INFO:root:skipping line #3440 with OOV words: Nairobi Kenya Paramaribo Suriname
INFO:root:skipping line #3633 with OOV words: Nuuk Greenland Quito Ecuador

recall of word2vec & mutiplication model: 0.7633


In [13]:
# Release this part of memory for the rest of models
del word2vec_model

In [14]:
# Load pre-trained glove model from dist
glove50d_model = glove2word2vec('glove.6B.50d.txt')

INFO:gensim.models.word2vec:loading projection weights from glove.6B.50d.word2vec.txt
INFO:gensim.models.word2vec:loaded (400000L, 50L) matrix from glove.6B.50d.word2vec.txt
INFO:gensim.models.word2vec:precomputing L2-norms of word weight vectors


In [15]:
print "recall of glove50d & addition model:", recallOfAnalogyModel('questions-words.txt', glove50d_model, 1, restrict_vocab = 200000)

INFO:root:correct pair:122, total pair:200
INFO:root:skipping line #5571 with OOV words: Macedonia denar Canada dollar
INFO:root:skipping line #5610 with OOV words: Malaysia ringgit Macedonia denar
INFO:root:correct pair:177, total pair:400
INFO:root:correct pair:238, total pair:600
INFO:root:correct pair:352, total pair:800


recall of glove50d & addition model: 0.4425


In [16]:
print "recall of glove50d & multiplication model:", recallOfAnalogyModel('questions-words.txt', glove50d_model, 2, restrict_vocab = 200000)

INFO:root:correct pair:98, total pair:200
INFO:root:skipping line #5571 with OOV words: Macedonia denar Canada dollar
INFO:root:skipping line #5610 with OOV words: Malaysia ringgit Macedonia denar
INFO:root:correct pair:131, total pair:400
INFO:root:correct pair:182, total pair:600
INFO:root:correct pair:276, total pair:800


recall of glove50d & multiplication model: 0.3501


In [17]:
del glove50d_model

In [18]:
# Load pre-trained glove model from dist
glove100d_model = glove2word2vec('glove.6B.100d.txt')

INFO:gensim.models.word2vec:loading projection weights from glove.6B.100d.word2vec.txt
INFO:gensim.models.word2vec:loaded (400000L, 100L) matrix from glove.6B.100d.word2vec.txt
INFO:gensim.models.word2vec:precomputing L2-norms of word weight vectors


In [19]:
print "recall of glove100d & addition model:", recallOfAnalogyModel('questions-words.txt', glove100d_model, 1, restrict_vocab = 200000)

INFO:root:correct pair:167, total pair:200
INFO:root:skipping line #5571 with OOV words: Macedonia denar Canada dollar
INFO:root:skipping line #5610 with OOV words: Malaysia ringgit Macedonia denar
INFO:root:correct pair:262, total pair:400
INFO:root:correct pair:362, total pair:600
INFO:root:correct pair:511, total pair:800


recall of glove100d & addition model: 0.6335


In [20]:
print "recall of glove100d & multiplication model:", recallOfAnalogyModel('questions-words.txt', glove100d_model, 2, restrict_vocab = 200000)

INFO:root:correct pair:156, total pair:200
INFO:root:skipping line #5571 with OOV words: Macedonia denar Canada dollar
INFO:root:skipping line #5610 with OOV words: Malaysia ringgit Macedonia denar
INFO:root:correct pair:243, total pair:400
INFO:root:correct pair:336, total pair:600
INFO:root:correct pair:473, total pair:800


recall of glove100d & multiplication model: 0.5945


In [21]:
del glove100d_model

In [22]:
# Load pre-trained glove model from dist
glove200d_model = glove2word2vec('glove.6B.200d.txt')

INFO:gensim.models.word2vec:loading projection weights from glove.6B.200d.word2vec.txt
INFO:gensim.models.word2vec:loaded (400000L, 200L) matrix from glove.6B.200d.word2vec.txt
INFO:gensim.models.word2vec:precomputing L2-norms of word weight vectors


In [23]:
print "recall of glove200d & addition model:", recallOfAnalogyModel('questions-words.txt', glove200d_model, 1, restrict_vocab = 200000)

INFO:root:correct pair:181, total pair:200
INFO:root:skipping line #5571 with OOV words: Macedonia denar Canada dollar
INFO:root:skipping line #5610 with OOV words: Malaysia ringgit Macedonia denar
INFO:root:correct pair:291, total pair:400
INFO:root:correct pair:405, total pair:600
INFO:root:correct pair:564, total pair:800


recall of glove200d & addition model: 0.6951


In [24]:
print "recall of glove200d & multiplication model:", recallOfAnalogyModel('questions-words.txt', glove200d_model, 2, restrict_vocab = 200000)

INFO:root:correct pair:177, total pair:200
INFO:root:skipping line #5571 with OOV words: Macedonia denar Canada dollar
INFO:root:skipping line #5610 with OOV words: Malaysia ringgit Macedonia denar
INFO:root:correct pair:282, total pair:400
INFO:root:correct pair:393, total pair:600
INFO:root:correct pair:553, total pair:800


recall of glove200d & multiplication model: 0.6858


In [25]:
del glove200d_model

In [26]:
# Load pre-trained glove model from dist
glove300d_model = glove2word2vec('glove.6B.300d.txt')

INFO:gensim.models.word2vec:loading projection weights from glove.6B.300d.word2vec.txt
INFO:gensim.models.word2vec:loaded (400000L, 300L) matrix from glove.6B.300d.word2vec.txt
INFO:gensim.models.word2vec:precomputing L2-norms of word weight vectors


In [27]:
print "recall of glove300d & addition model:", recallOfAnalogyModel('questions-words.txt', glove300d_model, 1, restrict_vocab = 200000)

INFO:root:correct pair:183, total pair:200
INFO:root:skipping line #5571 with OOV words: Macedonia denar Canada dollar
INFO:root:skipping line #5610 with OOV words: Malaysia ringgit Macedonia denar
INFO:root:correct pair:300, total pair:400
INFO:root:correct pair:421, total pair:600
INFO:root:correct pair:584, total pair:800


recall of glove300d & addition model: 0.7136


In [28]:
print "recall of glove300d & multiplication model:", recallOfAnalogyModel('questions-words.txt', glove300d_model, 2, restrict_vocab = 200000)

INFO:root:correct pair:183, total pair:200
INFO:root:skipping line #5571 with OOV words: Macedonia denar Canada dollar
INFO:root:skipping line #5610 with OOV words: Malaysia ringgit Macedonia denar
INFO:root:correct pair:303, total pair:400
INFO:root:correct pair:428, total pair:600
INFO:root:correct pair:594, total pair:800


recall of glove300d & multiplication model: 0.732


In [29]:
del glove300d_model